# MNLP Homework 3 NLI task with Huggingface-Trainer


## Initialization

In [ ]:
#@title Global config
# language_model_name = "distilbert-base-uncased"
# language_model_name = "bert-base-uncased"
language_model_name = "roberta-base"

augmentation_dir = "./augmentation"
%mkdir -p {augmentation_dir}
rand_index = 42

train_baseline = False

In [ ]:
#@title Packages installation
!pip install -q accelerate -U
!pip install -q datasets
!pip install -q wandb
!pip install -q torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have reques

In [ ]:
#@title General imports
import os
import re
import string
import random
import matplotlib.pyplot as plt

import torch
import wandb
import numpy as np
import pandas as pd
from typing import Dict
from datasets import (
    load_dataset,
    load_metric,
    load_from_disk,
    concatenate_datasets
)
from torchinfo import summary
from tqdm.notebook import tqdm

import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

# set seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
random.seed(42)

project_name = "nli-baseline"
wandb.login(key="4f60a58a3df8475767359f2cd20a52603629f148")

[nltk_data] Downloading package wordnet to /root/nltk_data...
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#@title Huggingface imports

from transformers import DataCollatorWithPadding
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    set_seed,
    TrainerCallback
)
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
set_seed(42)

## Fever Dataset

The dataset is structured as follows:
fever_dataset['split_name'][raw_index]['column_key']

where:
    
*   'split_name' is a sting in ['train', 'test', 'validation']
*   raw_index is a integer
*   'colum_key' is the name of the attributes and can be ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl']




In [ ]:
# load a chunk of FEVER dataset
fever_dataset = load_dataset("tommasobonomo/sem_augmented_fever_nli", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/51086 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2288 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2287 [00:00<?, ? examples/s]

In [ ]:
# structure of the dataset
fever_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 51086
    })
    validation: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2288
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2287
    })
})

In [ ]:
# some examples
rand_index = random.randint(0, len(fever_dataset['train']))
print(f"Premise: {fever_dataset['train'][rand_index]['premise']}")
print(f"Hypothesis: {fever_dataset['train'][rand_index]['hypothesis']}")
print(f"Label: {fever_dataset['train'][rand_index]['label']}")
print(f"WSD: {fever_dataset['train'][rand_index]['wsd']}")
print(f"SRL: {fever_dataset['train'][rand_index]['srl']}")

Premise: Lee Harvey Oswald ( October 18 , 1939 -- November 24 , 1963 ) was an American former U.S. Marine who was arrested for the assassination of United States President John F. Kennedy on November 22 , 1963 . Oswald shot and killed Kennedy as the President traveled by motorcade through Dealey Plaza in the city of Dallas , Texas . Following Kennedy 's assassination , Oswald was initially arrested for the murder of police officer J. D. Tippit , who was killed on a Dallas street about 45 minutes after Kennedy was shot .
Hypothesis: Lee Harvey Oswald assassinated someone.
Label: ENTAILMENT
WSD: {'premise': [{'index': 0, 'text': 'Lee', 'pos': 'PROPN', 'lemma': 'Lee', 'bnSynsetId': 'O', 'wnSynsetOffset': 'O', 'nltkSynset': 'O'}, {'index': 1, 'text': 'Harvey', 'pos': 'PROPN', 'lemma': 'Harvey', 'bnSynsetId': 'O', 'wnSynsetOffset': 'O', 'nltkSynset': 'O'}, {'index': 2, 'text': 'Oswald', 'pos': 'PROPN', 'lemma': 'Oswald', 'bnSynsetId': 'O', 'wnSynsetOffset': 'O', 'nltkSynset': 'O'}, {'index'

In [ ]:
# adversarial fever dataset
adversarial_testset = load_dataset("iperbole/adversarial_fever_nli", trust_remote_code=True)

Generating test split:   0%|          | 0/337 [00:00<?, ? examples/s]

In [ ]:
# TODO: load adversarial augmentation here
#@title Augmentation

# !rm -rf {augmentation_dir}

# download movie augmentation
if not os.path.exists(augmentation_dir + '/movie'):
    %mkdir -p {augmentation_dir}/movie
    !wget -q https://github.com/leeoos/airo-nli/raw/main/augmentation/datasets/movie/data-00000-of-00001.arrow -O {augmentation_dir}/movie/data-00000-of-00001.arrow
    !wget -q https://github.com/leeoos/airo-nli/raw/main/augmentation/datasets/movie/dataset_info.json -O {augmentation_dir}/movie/dataset_info.json
    !wget -q https://github.com/leeoos/airo-nli/raw/main/augmentation/datasets/movie/state.json -O {augmentation_dir}/movie/state.json

# download marrige augmentation
if not os.path.exists(augmentation_dir + '/marriage'):
    %mkdir -p {augmentation_dir}/marriage
    !wget -q https://github.com/leeoos/airo-nli/raw/main/augmentation/datasets/marriage/data-00000-of-00001.arrow -O {augmentation_dir}/marriage/data-00000-of-00001.arrow
    !wget -q https://github.com/leeoos/airo-nli/raw/main/augmentation/datasets/marriage/dataset_info.json -O {augmentation_dir}/marriage/dataset_info.json
    !wget -q https://github.com/leeoos/airo-nli/raw/main/augmentation/datasets/marriage/state.json -O {augmentation_dir}/marriage/state.json

# download born augmentation
if not os.path.exists(augmentation_dir + '/born'):
    %mkdir -p {augmentation_dir}/born
    !wget -q https://github.com/leeoos/airo-nli/raw/main/augmentation/datasets/born/data-00000-of-00001.arrow -O {augmentation_dir}/born/data-00000-of-00001.arrow
    !wget -q https://github.com/leeoos/airo-nli/raw/main/augmentation/datasets/born/dataset_info.json -O {augmentation_dir}/born/dataset_info.json
    !wget -q https://github.com/leeoos/airo-nli/raw/main/augmentation/datasets/born/state.json -O {augmentation_dir}/born/state.json

# download synonym augmentation
if not os.path.exists(augmentation_dir + '/syn'):
    %mkdir -p {augmentation_dir}/syn
    !wget -q https://github.com/leeoos/airo-nli/raw/main/augmentation/datasets/syn/data-00000-of-00001.arrow -O {augmentation_dir}/syn/data-00000-of-00001.arrow
    !wget -q https://github.com/leeoos/airo-nli/raw/main/augmentation/datasets/syn/dataset_info.json -O {augmentation_dir}/syn/dataset_info.json
    !wget -q https://github.com/leeoos/airo-nli/raw/main/augmentation/datasets/syn/state.json -O {augmentation_dir}/syn/state.json


# load augmentation
movie_augmentation = load_from_disk(augmentation_dir + '/movie')
marriage_augmentation = load_from_disk(augmentation_dir + '/marriage')
born_augmentation = load_from_disk(augmentation_dir + '/born')
syn_augmentation = load_from_disk(augmentation_dir + '/syn')

# uncomment the
list_of_augmentation = [
    fever_dataset['train'],
    movie_augmentation,
    marriage_augmentation,
    born_augmentation,
    syn_augmentation
]
augmented_dataset = concatenate_datasets(list_of_augmentation)


print(f"Augmented dataset type: {type(augmented_dataset)}")
print(f"Train split length: {len(fever_dataset['train'])}")
print(f"Train split augmentated: {len(augmented_dataset)}")
print(f"Addeded samples: {len(augmented_dataset) - len(fever_dataset['train'])}")
augmenetd_dataset = augmented_dataset.shuffle(seed=42)


Augmented dataset type: <class 'datasets.arrow_dataset.Dataset'>
Train split length: 51086
Train split augmentated: 59578
Addeded samples: 8492


## Tokenization

In [ ]:
#@title Tokenizer initialization

tokenizer = AutoTokenizer.from_pretrained(language_model_name)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

label_map = {
    'ENTAILMENT': 0,
    'NEUTRAL': 1,
    'CONTRADICTION': 2,
    'NOT ENOUGH INFO': None
}

def tokenize_function(examples):
    examples['label'] = [label_map[label] for label in examples['label']]
    return tokenizer(examples['premise'], examples['hypothesis'], padding=True, truncation=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
#@title Fever dataset tokenization

print(f"Tokenize fever dataset...")
tokenized_fever_datasets = fever_dataset.map(tokenize_function, batched=True)

# tokenized fever dataset
print(f" dataset after tokenization: {tokenized_fever_datasets}")
print(f"Input IDs: {tokenized_fever_datasets['train'][rand_index]['input_ids']}")
print(f"Attention Mask: {tokenized_fever_datasets['train'][rand_index]['attention_mask']}")

Tokenize fever dataset...


Map:   0%|          | 0/51086 [00:00<?, ? examples/s]

Map:   0%|          | 0/2288 [00:00<?, ? examples/s]

Map:   0%|          | 0/2287 [00:00<?, ? examples/s]

 dataset after tokenization: DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl', 'input_ids', 'attention_mask'],
        num_rows: 51086
    })
    validation: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl', 'input_ids', 'attention_mask'],
        num_rows: 2288
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl', 'input_ids', 'attention_mask'],
        num_rows: 2287
    })
})
Input IDs: [0, 24403, 4641, 35032, 36, 779, 504, 2156, 28234, 480, 759, 706, 2156, 18733, 4839, 21, 41, 470, 320, 121, 4, 104, 4, 6144, 54, 21, 1128, 13, 5, 16351, 9, 315, 532, 270, 610, 274, 4, 5076, 15, 759, 820, 2156, 18733, 479, 35032, 738, 8, 848, 5076, 25, 5, 270, 8468, 30, 4243, 26671, 149, 926, 11331, 11984, 11, 5, 343, 9, 3160, 2156, 1184, 479, 3515, 5076, 128, 29, 16351, 2156, 35032, 21, 3225, 1128, 13, 5, 1900, 9, 249, 1036, 344, 4, 211, 4, 255, 5600, 405, 2156, 54,

In [ ]:
#@title Adversarial dataset tokenization
print(f"Tokenize adversarial dataset...")
tokenized_adversarial_testsets = adversarial_testset.map(tokenize_function, batched=True)

Tokenize adversarial dataset...


Map:   0%|          | 0/337 [00:00<?, ? examples/s]

In [ ]:
print(f"Tokenize augmented dataset...")
tokenized_augmented_dataset = augmented_dataset.map(tokenize_function, batched=True)

Tokenize augmented dataset...


Map:   0%|          | 0/59578 [00:00<?, ? examples/s]

## Evaluation Metrics

In [ ]:
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy", trust_remote_code=True)
    load_f1 = load_metric("f1", trust_remote_code=True)
    load_precision = load_metric("precision", trust_remote_code=True)
    load_recall = load_metric("recall", trust_remote_code=True)

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    precision = load_precision.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = load_recall.compute(predictions=predictions, references=labels, average="weighted")["recall"]

    return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}


In [ ]:
def pretty_print_dict(d, indent=0):
    for key, value in d.items():
        print(' ' * indent + str(key) + ':', end=' ')
        if isinstance(value, dict):
            print()
            pretty_print_dict(value, indent + 4)
        else:
            print(value)


In [ ]:
class CustomCallback(TrainerCallback):
    def __init__(self, name=''):
        self.name = name
        self.best_metric = -float('inf')
        self.best_model_checkpoint = None

    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero and 'eval_loss' in logs:
            step = state.global_step
            train_loss = logs.get('loss', 0)
            eval_loss = logs['eval_loss']
            accuracy = logs['eval_accuracy']
            f1 = logs['eval_f1']
            precision = logs['eval_precision']
            recall = logs['eval_recall']

            if accuracy > self.best_metric:
                self.best_metric = eval_loss
                self.best_model_checkpoint = f"training_dir/checkpoint-{self.name}-{step}"

            # print(f"{step:>5} {train_loss:.4f} {eval_loss:.4f} {accuracy:.4f} {f1:.4f} {precision:.4f} {recall:.4f}")

    def on_train_begin(self, args, state, control, **kwargs):
        print("Step  Train   Eval    Acc     F1      Prec    Recall")
        print("----- ------- ------- ------- ------- ------- -------")


# Architecture

In [ ]:
#@title Hyperparameters

# batch size
batch_size = 32

# optimization
learning_rate = 1e-4
weight_decay = 0.001

# training
epochs = 1
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

set_seed(42)

Using cuda device


## Baseline

In [ ]:
#@title Baseline Training


baseline_model = AutoModelForSequenceClassification.from_pretrained(
    language_model_name,
    ignore_mismatched_sizes=True,
    output_attentions=False,
    output_hidden_states=False,
    num_labels=3 # number of the classes
)

# baseline_model summary
summary(baseline_model, input_size=(batch_size, 512), dtypes=[torch.long])

baseline_training_args = TrainingArguments(
    report_to=["wandb"],
    run_name="baseline",
    output_dir="training_dir/baseline",           # output directory [Mandatory]
    num_train_epochs=epochs,                      # total number of training epochs
    per_device_train_batch_size=batch_size,       # batch size per device during training
    warmup_steps=1500,                            # number of warmup steps for learning rate scheduler
    weight_decay=weight_decay,                    # strength of weight decay
    learning_rate=learning_rate,                  # learning rate
    evaluation_strategy="steps",
    eval_steps=500,                               # Evaluate every 500 steps
    save_strategy="steps",
    save_steps=500,                               # Save model every 500 steps
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
)

wandb.init(project=project_name, name="baseline-run")

baseline_trainer = Trainer(
   model=baseline_model,
   args=baseline_training_args,
   train_dataset=tokenized_fever_datasets["train"],
   eval_dataset=tokenized_fever_datasets["validation"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
   callbacks=[CustomCallback(name='baseline')],
)

# let's train ...
if train_baseline: baseline_trainer.train()
baseline_trainer.train()
wandb.finish()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: Currently logged in as: leonardo-colosi (sairo-team). Use `wandb login --relogin` to force relogin


Step  Train   Eval    Acc     F1      Prec    Recall
----- ------- ------- ------- ------- ------- -------


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.703100,0.831586,0.713724,0.695321,0.723970,0.713724
1000,0.459600,0.737381,0.727273,0.721433,0.725297,0.727273
1500,0.485800,0.725080,0.703671,0.698721,0.721620,0.703671


<ipython-input-14-0d856be898a5>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy", trust_remote_code=True)


eval/accuracy,▄█▁
eval/f1,▁█▂
eval/loss,█▂▁
eval/precision,▅█▁
eval/recall,▄█▁
eval/runtime,█▁▃
eval/samples_per_second,▁█▆
eval/steps_per_second,▁█▆
train/epoch,▁▁▄▄▇▇█
train/global_step,▁▁▄▄▇▇█
train/grad_norm,█▁█


In [ ]:
#@title Baseline Evaluation

# on test
wandb.init(project=project_name, name="baseline-test")
test_results = baseline_trainer.evaluate(eval_dataset=tokenized_fever_datasets["test"])
print("\nRegular Test Results:")
# pretty_print_dict(test_results)
wandb.finish()
print()

# on adversarial
wandb.init(project=project_name, name="baseline-adversarial")
adversarial_results = baseline_trainer.evaluate(eval_dataset=tokenized_adversarial_testsets["test"])
print("\nAdversarial Test Results:")
wandb.finish()
# pretty_print_dict(adversarial_results)



Regular Test Results:


eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.71885



Adversarial Test Results:


eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.51929


## Augmented Baseline

In [ ]:
#@title Model initialization

model_roberta = AutoModelForSequenceClassification.from_pretrained(
    language_model_name,
    ignore_mismatched_sizes=True,
    output_attentions=False,
    output_hidden_states=False,
    num_labels=3 # number of the classes
)

summary(model_roberta, input_size=(batch_size, 512), dtypes=[torch.long])

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Layer (type:depth-idx)                                       Output Shape              Param #
RobertaForSequenceClassification                             [32, 3]                   --
├─RobertaModel: 1-1                                          [32, 512, 768]            --
│    └─RobertaEmbeddings: 2-1                                [32, 512, 768]            --
│    │    └─Embedding: 3-1                                   [32, 512, 768]            38,603,520
│    │    └─Embedding: 3-2                                   [32, 512, 768]            768
│    │    └─Embedding: 3-3                                   [32, 512, 768]            394,752
│    │    └─LayerNorm: 3-4                                   [32, 512, 768]            1,536
│    │    └─Dropout: 3-5                                     [32, 512, 768]            --
│    └─RobertaEncoder: 2-2                                   [32, 512, 768]            --
│    │    └─ModuleList: 3-6                                  --               

In [ ]:
#@title Model Trainer

training_args = TrainingArguments(
    report_to=["wandb"],
    run_name="augmented_baseline",
    output_dir="training_dir/roberta",            # output directory [Mandatory]
    num_train_epochs=epochs,                      # total number of training epochs
    per_device_train_batch_size=batch_size,       # batch size per device during training
    warmup_steps=1500,                            # number of warmup steps for learning rate scheduler
    weight_decay=weight_decay,                    # strength of weight decay
    learning_rate=learning_rate,                  # learning rate
    evaluation_strategy="steps",
    eval_steps=500,                               # Evaluate every 500 steps
    save_strategy="steps",
    save_steps=500,                               # Save model every 500 steps
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
#@title Train Augmented

wandb.init(project=project_name, name="augmented-baseline-run")

roberta_trainer = Trainer(
   model=model_roberta,
   args=training_args,
   train_dataset=tokenized_augmented_dataset,
   eval_dataset=tokenized_fever_datasets["validation"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
   callbacks=[CustomCallback(name='roberta')],
)

# let's train ...
roberta_trainer.train()
wandb.finish()

Step  Train   Eval    Acc     F1      Prec    Recall
----- ------- ------- ------- ------- ------- -------


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.766900,0.938222,0.704983,0.697093,0.714975,0.704983
1000,0.508300,0.848273,0.717657,0.704292,0.720264,0.717657
1500,0.506300,0.808042,0.700612,0.678852,0.703088,0.700612


eval/accuracy,▃█▁
eval/f1,▆█▁
eval/loss,█▃▁
eval/precision,▆█▁
eval/recall,▃█▁
eval/runtime,█▁▄
eval/samples_per_second,▁█▅
eval/steps_per_second,▁█▅
train/epoch,▁▁▄▄▆▆█
train/global_step,▁▁▄▄▆▆█
train/grad_norm,█▂▁


In [ ]:
#@title Augmented Evaluation

# on test
wandb.init(project=project_name, name="augmented-baseline-test")
test_results = roberta_trainer.evaluate(eval_dataset=tokenized_fever_datasets["test"])
print("\nRegular Test Results:")
# pretty_print_dict(test_results)
wandb.finish()
print()

# on adversarial
wandb.init(project=project_name, name="augmented-baseline-adversarial")
adversarial_results = roberta_trainer.evaluate(eval_dataset=tokenized_adversarial_testsets["test"])
print("\nAdversarial Test Results:")
# pretty_print_dict(adversarial_results)
wandb.finish()


Regular Test Results:


eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.69786



Adversarial Test Results:


eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.53709


# Ensabled Architecture

In [ ]:
#@title Ensamble Metrics

def compute_ens_metrics(ensemble_predictions, labels):
    accuracy_metric = load_metric("accuracy")
    f1_metric = load_metric("f1")
    precision_metric = load_metric("precision")
    recall_metric = load_metric("recall")

    accuracy = accuracy_metric.compute(predictions=ensemble_predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=ensemble_predictions, references=labels, average="weighted")["f1"]
    precision = precision_metric.compute(predictions=ensemble_predictions, references=labels, average="weighted")["precision"]
    recall = recall_metric.compute(predictions=ensemble_predictions, references=labels, average="weighted")["recall"]

    return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
#@title BERT

# load bert tokenizer and collator
tokenizer_bert = AutoTokenizer.from_pretrained("bert-base-uncased")
data_collator_bert = DataCollatorWithPadding(tokenizer=tokenizer_bert)

model_bert = AutoModelForSequenceClassification.from_pretrained(
    language_model_name,
    ignore_mismatched_sizes=True,
    output_attentions=False,
    output_hidden_states=False,
    num_labels=3 # number of the classes
)

summary(model_bert, input_size=(batch_size, 512), dtypes=[torch.long])

training_args_bert = TrainingArguments(
    report_to=["wandb"],
    run_name="bert_ensambled",
    output_dir="training_dir/ensambled",          # output directory [Mandatory]
    num_train_epochs=epochs,                      # total number of training epochs
    per_device_train_batch_size=batch_size,       # batch size per device during training
    warmup_steps=1500,                            # number of warmup steps for learning rate scheduler
    weight_decay=weight_decay,                    # strength of weight decay
    learning_rate=learning_rate,                  # learning rate
    evaluation_strategy="steps",
    eval_steps=500,                               # Evaluate every 500 steps
    save_strategy="steps",
    save_steps=500,                               # Save model every 500 steps
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
)

wandb.init(project=project_name, name="ensambled-bert-run")

trainer_bert = Trainer(
    model=model_bert,
    args=training_args_bert,
    train_dataset=tokenized_fever_datasets["train"],
    eval_dataset=tokenized_fever_datasets["validation"],
    tokenizer=tokenizer_bert,
    data_collator=data_collator_bert,
    compute_metrics=compute_metrics,
    callbacks=[CustomCallback(name='bert')],
)
trainer_bert.train()
wandb.finish()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step  Train   Eval    Acc     F1      Prec    Recall
----- ------- ------- ------- ------- ------- -------


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.693900,0.789250,0.727710,0.713001,0.727889,0.727710
1000,0.458300,0.812458,0.713287,0.708827,0.722980,0.713287
1500,0.483000,0.777240,0.694056,0.681924,0.714032,0.694056


eval/accuracy,█▅▁
eval/f1,█▇▁
eval/loss,▃█▁
eval/precision,█▆▁
eval/recall,█▅▁
eval/runtime,▁█▃
eval/samples_per_second,█▁▆
eval/steps_per_second,█▁▆
train/epoch,▁▁▄▄▇▇█
train/global_step,▁▁▄▄▇▇█
train/grad_norm,█▁▂


In [ ]:
#@title Test BERT

wandb.init(project=project_name, name="test-bert")
test_results = trainer_bert.evaluate(eval_dataset=tokenized_fever_datasets["test"])
print("\nRegular Test Results:")
# pretty_print_dict(test_results)
wandb.finish()
print()

# on adversarial
wandb.init(project=project_name, name="baseline-adversarial")
adversarial_results = trainer_bert.evaluate(eval_dataset=tokenized_adversarial_testsets["test"])
print("\nAdversarial Test Results:")
wandb.finish()
# pretty_print_dict(adversarial_results)


Regular Test Results:


eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.69961



Adversarial Test Results:


eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.52522


In [ ]:
#@title Ensambled Evaluation

# Test
# get predictions from RoBERTa
predictions_roberta = baseline_trainer.predict(tokenized_fever_datasets["test"]).predictions
probs_roberta = torch.nn.functional.softmax(torch.tensor(predictions_roberta), dim=-1).numpy()

# get predictions from BERT
predictions_bert = trainer_bert.predict(tokenized_fever_datasets["test"]).predictions
probs_bert = torch.nn.functional.softmax(torch.tensor(predictions_bert), dim=-1).numpy()

# average the probabilities
average_probs = (probs_roberta + probs_bert) / 3
ensemble_predictions = np.argmax(average_probs, axis=-1)

# compute metrics
labels = tokenized_fever_datasets["test"]["label"]
print("\nEnsable method Test ")
pretty_print_dict(compute_ens_metrics(ensemble_predictions, labels))


# Adversarial
# get predictions from RoBERTa
predictions_roberta = baseline_trainer.predict(tokenized_adversarial_testsets["test"]).predictions
probs_roberta = torch.nn.functional.softmax(torch.tensor(predictions_roberta), dim=-1).numpy()

# get predictions from BERT
predictions_bert = trainer_bert.predict(tokenized_adversarial_testsets["test"]).predictions
probs_bert = torch.nn.functional.softmax(torch.tensor(predictions_bert), dim=-1).numpy()

# average the probabilities
average_probs = (probs_roberta + probs_bert) / 3
ensemble_predictions = np.argmax(average_probs, axis=-1)

# compute metrics
labels = tokenized_adversarial_testsets["test"]["label"]
print("\nEnsable method Adversarial ")
pretty_print_dict(compute_ens_metrics(ensemble_predictions, labels))


Ensable method Test 
accuracy: 0.7166593790992567
f1: 0.7047458138290743
precision: 0.7123130173480603
recall: 0.7166593790992567



Ensable method Adversarial 
accuracy: 0.5400593471810089
f1: 0.5402649947147933
precision: 0.5541920870316394
recall: 0.5400593471810089


In [ ]:
#@title Train BERT on Augmentation

# load bert tokenizer and collator
tokenizer_bert = AutoTokenizer.from_pretrained("bert-base-uncased")
data_collator_bert = DataCollatorWithPadding(tokenizer=tokenizer_bert)

model_bert = AutoModelForSequenceClassification.from_pretrained(
    language_model_name,
    ignore_mismatched_sizes=True,
    output_attentions=False,
    output_hidden_states=False,
    num_labels=3 # number of the classes
)

summary(model_bert, input_size=(batch_size, 512), dtypes=[torch.long])

training_args_bert = TrainingArguments(
    report_to=["wandb"],
    run_name="bert_ensambled",
    output_dir="training_dir/ensambled",          # output directory [Mandatory]
    num_train_epochs=epochs,                      # total number of training epochs
    per_device_train_batch_size=batch_size,       # batch size per device during training
    warmup_steps=1500,                            # number of warmup steps for learning rate scheduler
    weight_decay=weight_decay,                    # strength of weight decay
    learning_rate=learning_rate,                  # learning rate
    evaluation_strategy="steps",
    eval_steps=500,                               # Evaluate every 500 steps
    save_strategy="steps",
    save_steps=500,                               # Save model every 500 steps
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
)

wandb.init(project=project_name, name="ensambled-run")

trainer_bert = Trainer(
    model=model_bert,
    args=training_args_bert,
    train_dataset=tokenized_augmented_dataset,
    eval_dataset=tokenized_fever_datasets["validation"],
    tokenizer=tokenizer_bert,
    data_collator=data_collator_bert,
    compute_metrics=compute_metrics,
    callbacks=[CustomCallback(name='model')],
)
trainer_bert.train()
wandb.finish()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step  Train   Eval    Acc     F1      Prec    Recall
----- ------- ------- ------- ------- ------- -------


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.750200,0.852997,0.715035,0.710235,0.716405,0.715035
1000,0.504600,0.786446,0.720717,0.712721,0.730086,0.720717
1500,0.504400,0.793009,0.703671,0.683004,0.703770,0.703671


eval/accuracy,▆█▁
eval/f1,▇█▁
eval/loss,█▁▂
eval/precision,▄█▁
eval/recall,▆█▁
eval/runtime,▁█▃
eval/samples_per_second,█▁▆
eval/steps_per_second,█▁▆
train/epoch,▁▁▄▄▆▆█
train/global_step,▁▁▄▄▆▆█
train/grad_norm,█▃▁


In [ ]:
#@title Test BERT on Augmentation

wandb.init(project=project_name, name="augmented-bert")
test_results = trainer_bert.evaluate(eval_dataset=tokenized_fever_datasets["test"])
print("\nRegular Test Results:")
# pretty_print_dict(test_results)
wandb.finish()
print()

# on adversarial
wandb.init(project=project_name, name="baseline-adversarial")
adversarial_results = trainer_bert.evaluate(eval_dataset=tokenized_adversarial_testsets["test"])
print("\nAdversarial Test Results:")
wandb.finish()
# pretty_print_dict(adversarial_results)


Regular Test Results:


eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.69436



Adversarial Test Results:


eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.51039


In [ ]:
#@title Augmented Ensabled Evaluation

# Test
# get predictions from RoBERTa
predictions_roberta = roberta_trainer.predict(tokenized_fever_datasets["test"]).predictions
probs_roberta = torch.nn.functional.softmax(torch.tensor(predictions_roberta), dim=-1).numpy()

# get predictions from BERT
predictions_bert = trainer_bert.predict(tokenized_fever_datasets["test"]).predictions
probs_bert = torch.nn.functional.softmax(torch.tensor(predictions_bert), dim=-1).numpy()

# average the probabilities
average_probs = (probs_roberta + probs_bert) / 3
ensemble_predictions = np.argmax(average_probs, axis=-1)

# compute metrics
labels = tokenized_fever_datasets["test"]["label"]
print("\nEnsable method Test ")
pretty_print_dict(compute_ens_metrics(ensemble_predictions, labels))


# Adversarial
# get predictions from RoBERTa
predictions_roberta = roberta_trainer.predict(tokenized_adversarial_testsets["test"]).predictions
probs_roberta = torch.nn.functional.softmax(torch.tensor(predictions_roberta), dim=-1).numpy()

# get predictions from BERT
predictions_bert = trainer_bert.predict(tokenized_adversarial_testsets["test"]).predictions
probs_bert = torch.nn.functional.softmax(torch.tensor(predictions_bert), dim=-1).numpy()

# average the probabilities
average_probs = (probs_roberta + probs_bert) / 3
ensemble_predictions = np.argmax(average_probs, axis=-1)

# compute metrics
labels = tokenized_adversarial_testsets["test"]["label"]
print("\nEnsable method Adversarial ")
pretty_print_dict(compute_ens_metrics(ensemble_predictions, labels))



Ensable method Test 
accuracy: 0.7004809794490598
f1: 0.6888408149663585
precision: 0.7017999386234032
recall: 0.7004809794490598



Ensable method Adversarial 
accuracy: 0.5192878338278932
f1: 0.5193730653075784
precision: 0.5398825302623522
recall: 0.5192878338278932
